# Simple Hello World example for IBM Cloud Functions PyWren

This is a simple Hello World example, showing how to take a function and run it with pywren. First we import the necessary libraries. 

In [1]:
import pywren_ibm_cloud as pywren
import numpy as np
import os

from IPython.display import display

Pywren is designed to run any existing python functions you have, in parallel, at scale, on the cloud. So first, we create an example python function.

In [2]:
def my_function(x):
    return x + 7

PyWren needs the keys to access IBM Cloud Object Storage and IBM Cloud Functions.

In [3]:
config = {'pywren' : { 
    'storage_bucket': 'STORAGE BUCKET',
    'storage_prefix': 'DIRECTORY IN THE BUCKET' },
 'ibm_cf': {
    'endpoint': 'https://openwhisk.ng.bluemix.net', 
    'namespace': 'YOUR IBM CLOUD FUNCTIONS NAMESPASE', 
    'api_key': 'YOUR IBM CLOUD FUNCTIONSAPI KEY'}, 
 'ibm_cos': {
    'endpoint': 'COS ENDPOUT, for example: http://s3-api.us-geo.objectstorage.softlayer.net', 
    'api_key' : 'COS API KEY'}}

To start using `pywren`, we first create an executor with the previous config.

In [4]:
wrenexec = pywren.ibm_cf_executor(config=config)

IBM Cloud Functions init for namespace: gilv@il.ibm.com_Test and host: https://openwhisk.ng.bluemix.net
IBM Cloud Functions init for runtime: pywren_3.6
IBM Cloud Functions executor created with ID c8a496b5-37ca


We can call `my_function(3)` remotely via `call_async`: 

In [5]:
wrenexec.call_async(my_function, 3)

Executor ID c8a496b5-37ca Uploading function and data
Executor ID c8a496b5-37ca Starting function invocation: my_function()
Executor ID c8a496b5-37ca Function 00000 - Activation ID: c7bda47f838e42c8bda47f838e42c89e - Time: 0.751 seconds
Executor ID c8a496b5-37ca Invocation done: 0.831 seconds


Future is a placeholder for the returned value from applying `my_function` to the number `3`. We can call `result` on it and get the result. Note that this will block until the remote job has completed. Once finished it calls `close` to clean all the unnecessary data stored in COS.

In [6]:
display(wrenexec.get_result())
wrenexec.clean()

    0%|          | 0/1  

Executor ID c8a496b5-37ca Getting result



  100%|██████████| 1/1  

10

Executor ID c8a496b5-37ca Cleaning partial results from PyWren bucket 'gilvdata'.
Executor ID c8a496b5-37ca Cleaning partial results for 'Apywren.jobs/c8a496b5-37ca'.
Executor ID c8a496b5-37ca Cleaning partial results for 'Apywren.jobs/c8a496b5-37ca'.


You can apply `my_function` to a list of arguments, and each will be executed remotely at the same time. 

In [7]:
wrenexec = pywren.ibm_cf_executor(config=config)
wrenexec.map(my_function, range(10))

The pywren `get_all_results` function will wait until all of the futures are done and return their results

In [8]:
results = wrenexec._get_all_results()
display(results)

  100%|██████████| 10/10  


[7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

That's it, we are now familiar how to make use of PyWren for parallelly executing a Python function across many actions in IBM Cloud Functions.